### Contents

* Guidelines for Prompting
  - Write clear and specific instructions 
  - Give the model time to "think"
  
* Iterative prompt development
* Capabilities 
  - Summarizing
  - Infering
  - **Transforming**
  - Expanding
* Building a Chatbot


# Transforming

이 노트북에서는 언어 번역, 맞춤법 및 문법 검사, 어조 조정 및 형식 변환과 같은 텍스트 변환 작업에 대규모 언어 모델을 사용하는 방법을 살펴봅니다.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 이것는 모델 출력의 무작위성 정도를 말합니다.[0~1]
    )
    return response.choices[0].message.content

## Translation

ChatGPT는 여러 언어로 된 소스로 교육을 받았습니다. 이렇게 하면 모델이 번역을 수행할 수 있습니다. 다음은 이 기능을 사용하는 방법에 대한 몇 가지 예입니다.

In [3]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría ordenar una licuadora.


In [4]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This is French.


In [5]:
prompt = f"""
Translate the following text to French and Spanish and\ 
English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: "Je veux commander un ballon de basket"

Spanish: "Quiero ordenar un balón de baloncesto"

English pirate: "I be wantin' to order a basketball"


In [6]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


### Universal Translator

대규모 다국적 전자 상거래 회사에서 IT를 담당하고 있다고 상상해보십시오. 
사용자는 모든 모국어로 IT 문제에 대해 메시지를 보내고 있습니다. 
귀하의 직원은 전 세계에서 왔으며 모국어만 구사합니다. 
만능 번역기가 필요합니다!

In [7]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                             # My screen is flashing
] 

In [8]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."

Korean: "내 모니터에는 빛나지 않는 픽셀이 있습니다." 

Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Korean: 내 마우스가 작동하지 않습니다 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken
Korean: 나의 Ctrl 키가 고장 났어요 

Original message (This is Chinese.): 我的屏幕在闪烁
English: My screen is flickering
Korean: 내 화면이 깜박거립니다 



## Try it yourself!
Try some translations on your own!

## Tone Transformation

글은 대상 독자에 따라 다를 수 있습니다. 
ChatGPT는 다양한 톤을 생성할 수 있습니다.

In [9]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention the specifications of the standing lamp. 

Sincerely,
Joe


## Format Conversion
ChatGPT는 형식 간에 번역할 수 있습니다. 프롬프트는 입력 및 출력 형식을 설명해야 합니다.

In [10]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<html>
<head>
  <title>Restaurant Employees</title>
</head>
<body>
  <table>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </table>
</body>
</html>


In [11]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

다음은 일반적인 문법 및 맞춤법 문제와 LLM의 응답에 대한 몇 가지 예입니다.

LLM에 텍스트 교정을 원한다는 신호를 보내려면 모델에 '교정' 또는 '교정 및 수정'을 지시합니다.

In [12]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they're suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
No errors found.
Their goes my freedom. There going to bring they're suitcases.

No errors found.

Rewritten:
Their goes my freedom. There going to bring their suitcases.
You're going to need your notebook.
No errors found.
No errors found


In [13]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room. Yes, adults also like pandas too. She takes \
it everywhere with her, and it's super soft and cute. One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price. It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [14]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I got </span>this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [15]:
text_kr = f"""
그녀가 내 방에서 내 것을 계속 가져가기 떼문에\
내 딸의 생일을 위해 이것을 얻었습니다.\
네, 어른들도 판다를 좋아합니다. 그녀는 그 어디든 가지고\
다니며 매우 부드럽고 귀엽다. 귀 중 하나가 다른 쪽보다\
약간 낮고 비대칭으로 설계되지 않은 것 같습니다.\
그래도 내가 지불 한 것에 비해 약간 적습니다.\
같은 가격에 더 큰 다른 옵션이 있을 수 있다고 생각합니다.\
예성보다 하루 일찍 도착해서 딸에게 주기 전에 직접 가지고 놀게 되었습니다.
"""
prompt = f"이 리뷰를 교정하고 수정하십시오.: ```{text_kr}```"
response_kr = get_completion(prompt)
print(response_kr)

그녀가 내 방에서 내 물건을 계속 가져가서, 내 딸의 생일을 위해 이것을 샀습니다. 네, 어른들도 판다를 좋아합니다. 그녀는 어디든지 가지고 다니며 매우 부드럽고 귀엽습니다. 귀 중 하나가 다른 쪽보다 약간 낮고 비대칭으로 설계된 것 같습니다. 그래도 내가 지불한 가격에 비해 약간 부족한 면이 있습니다. 같은 가격에 더 큰 다른 옵션이 있을 수 있다고 생각합니다. 예상보다 하루 일찍 도착해서 딸에게 주기 전에 직접 가지고 놀게 되었습니다.


In [16]:
diff = Redlines(text_kr,response_kr)
display(Markdown(diff.output_markdown))

그녀가 내 방에서 내 <span style='color:red;font-weight:700;text-decoration:line-through;'>것을 </span><span style='color:green;font-weight:700;'>물건을 </span>계속 <span style='color:red;font-weight:700;text-decoration:line-through;'>가져가기 떼문에내 </span><span style='color:green;font-weight:700;'>가져가서, 내 </span>딸의 생일을 위해 이것을 <span style='color:red;font-weight:700;text-decoration:line-through;'>얻었습니다.네, </span><span style='color:green;font-weight:700;'>샀습니다. 네, </span>어른들도 판다를 좋아합니다. 그녀는 <span style='color:red;font-weight:700;text-decoration:line-through;'>그 어디든 가지고다니며 </span><span style='color:green;font-weight:700;'>어디든지 가지고 다니며 </span>매우 부드럽고 <span style='color:red;font-weight:700;text-decoration:line-through;'>귀엽다. </span><span style='color:green;font-weight:700;'>귀엽습니다. </span>귀 중 하나가 다른 <span style='color:red;font-weight:700;text-decoration:line-through;'>쪽보다약간 </span><span style='color:green;font-weight:700;'>쪽보다 약간 </span>낮고 비대칭으로 <span style='color:red;font-weight:700;text-decoration:line-through;'>설계되지 않은 </span><span style='color:green;font-weight:700;'>설계된 </span>것 <span style='color:red;font-weight:700;text-decoration:line-through;'>같습니다.그래도 </span><span style='color:green;font-weight:700;'>같습니다. 그래도 </span>내가 <span style='color:red;font-weight:700;text-decoration:line-through;'>지불 한 것에 </span><span style='color:green;font-weight:700;'>지불한 가격에 </span>비해 약간 <span style='color:red;font-weight:700;text-decoration:line-through;'>적습니다.같은 </span><span style='color:green;font-weight:700;'>부족한 면이 있습니다. 같은 </span>가격에 더 큰 다른 옵션이 있을 수 있다고 <span style='color:red;font-weight:700;text-decoration:line-through;'>생각합니다.예성보다 </span><span style='color:green;font-weight:700;'>생각합니다. 예상보다 </span>하루 일찍 도착해서 딸에게 주기 전에 직접 가지고 놀게 되었습니다.

In [17]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush for my daughter's birthday to prevent her from constantly borrowing mine from my room. It's important to note that adults can also appreciate the charm of pandas. The plush is incredibly soft and cute, making it the perfect companion for my daughter wherever she goes. However, I did notice a slight asymmetry in the ears, which I believe was not intentional in the design. Additionally, considering the price I paid, I found the size to be a bit smaller than expected. I believe there may be larger options available for the same price point. Despite this, I was pleasantly surprised when the plush arrived a day earlier than anticipated, allowing me to enjoy it myself before gifting it to my daughter. Overall, while there are minor flaws, the quality and early delivery of the product make it a worthwhile purchase.

In [18]:
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I purchased </span>this <span style='color:green;font-weight:700;'>adorable panda plush </span>for my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for </span><span style='color:green;font-weight:700;'>daughter's birthday to prevent </span>her <span style='color:red;font-weight:700;text-decoration:line-through;'>birthday cuz she keeps taking </span><span style='color:green;font-weight:700;'>from constantly borrowing </span>mine from my room. <span style='color:red;font-weight:700;text-decoration:line-through;'>Yes, </span><span style='color:green;font-weight:700;'>It's important to note that </span>adults <span style='color:green;font-weight:700;'>can </span>also <span style='color:red;font-weight:700;text-decoration:line-through;'>like pandas too. She takes it everywhere with her, and it's super </span><span style='color:green;font-weight:700;'>appreciate the charm of pandas. The plush is incredibly </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute. One of </span><span style='color:green;font-weight:700;'>cute, making it </span>the <span style='color:red;font-weight:700;text-decoration:line-through;'>ears is </span><span style='color:green;font-weight:700;'>perfect companion for my daughter wherever she goes. However, I did notice a slight asymmetry in the ears, which I believe was not intentional in the design. Additionally, considering the price I paid, I found the size to be </span>a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>lower </span><span style='color:green;font-weight:700;'>smaller </span>than <span style='color:red;font-weight:700;text-decoration:line-through;'>the other, and </span><span style='color:green;font-weight:700;'>expected. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>don't think that was designed to </span><span style='color:green;font-weight:700;'>believe there may </span>be <span style='color:red;font-weight:700;text-decoration:line-through;'>asymmetrical. It's a bit small for what I paid for it though. I think there might be other </span><span style='color:green;font-weight:700;'>larger </span>options <span style='color:red;font-weight:700;text-decoration:line-through;'>that are bigger </span><span style='color:green;font-weight:700;'>available </span>for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price. It </span><span style='color:green;font-weight:700;'>price point. Despite this, I was pleasantly surprised when the plush </span>arrived a day earlier than <span style='color:red;font-weight:700;text-decoration:line-through;'>expected, so I got </span><span style='color:green;font-weight:700;'>anticipated, allowing me </span>to <span style='color:red;font-weight:700;text-decoration:line-through;'>play with </span><span style='color:green;font-weight:700;'>enjoy </span>it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>gifting </span>it to my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter.</span><span style='color:green;font-weight:700;'>daughter. Overall, while there are minor flaws, the quality and early delivery of the product make it a worthwhile purchase.</span>

## Try it yourself!
Try changing the instructions to form your own review.